In [1]:
# This chunk of code make it possible to use src functions
import sys
import os
module_path = os.path.abspath(os.path.join('../src'))

if module_path not in sys.path:
    sys.path.append(module_path)
import warnings
warnings.filterwarnings('ignore')

from dsproject import dsproject
from preprocess import *

dsp = dsproject()

In [2]:
from dsproject import min_date_train, max_date_train, min_date_test, max_date_test, train_label, test_label

In [1]:
min_date_text = '20160801'
max_date_text = '20160901'

In [ ]:
df = dsp.read_table('sor_ln_ar_filtered_' + min_date_text + '_' + max_date_text, 'preprocess', index_col=0)

In [ ]:
df0 = df

In [17]:
df = delete_missing_columns(df0)
df = delete_single_value_columns(df)

Deleted from missings list = ['CNTL_LMT_F']
Number of column deleted = 1
Delete single value column list
[]
Number of column deleted
0


In [18]:
df.head()

,HASH_IP_ID,HASH_AR_ID,POSN_DT,DLQ_DYS,CTR_AMT,OTSND_BAL_AMT,ACR_INT_AMT,PNP_AMT,EFF_INT_RATE,AUTO_DB_F,PRVN_CLT_AMT,INT_ARS_AMT,CST_LOAN_ST_TP1_ID,CST_LOAN_ST_TP2_ID,CLT_AMT,NPL_F
5,-7051478829698955641,3771768480733365957,2016-08-31,0.0,3344000.0,3086536.73,13398.23,3073138.50,15.00,Y,3086536.73,13398.23,1.0,1.0,3086536.73,N
7,1684987596721027991,4442653007279327772,2016-07-31,366.0,1500000.0,1237430.16,0.00,1237430.16,26.00,Y,0.00,0.00,0.0,5.0,0.00,Y
50,840676050197110366,1409159461470913354,2016-07-31,2.0,1400000.0,1260474.94,5830.49,1254644.45,15.00,Y,1260474.94,5830.49,1.0,1.0,1260474.94,N
71,-5396074436523461861,3223180271671636790,2016-08-31,638.0,1000000.0,944515.31,0.00,944515.31,26.00,Y,0.00,0.00,0.0,5.0,0.00,Y
82,-6170639285726162801,6773333778657135714,2016-07-31,0.0,205000.0,179029.61,113.66,178915.95,7.75,Y,0.00,113.66,1.0,1.0,0.00,N


In [19]:
target = dsp.read_table('group_payment_target', 'feature', use_schema=True)

In [21]:
df['HASH_AR_ID'] = df['HASH_AR_ID'].astype(str)
df['POSN_DT'] = parse_dates(df['POSN_DT'], '%Y-%m-%d')
prep = pd.merge(target[['HASH_AR_ID', 'FileDate', 'Previous_FileDate', 'Next_FileDate']], 
                df, on=['HASH_AR_ID'], how ='inner', suffixes=['','_y'])

HASH_AR_ID                   object
FileDate             datetime64[ns]
Previous_FileDate    datetime64[ns]
Next_FileDate        datetime64[ns]
dtype: object
HASH_AR_ID    object
POSN_DT       object
dtype: object
             HASH_AR_ID   FileDate Previous_FileDate Next_FileDate  \
0  -5086591535661867418 2015-10-07               NaT    2015-11-05   
1  -5086591535661867418 2015-10-07               NaT    2015-11-05   
2  -5086591535661867418 2015-10-07               NaT    2015-11-05   
3  -5086591535661867418 2015-11-05        2015-10-07    2015-12-08   
4  -5086591535661867418 2015-11-05        2015-10-07    2015-12-08   

            HASH_IP_ID    POSN_DT  DLQ_DYS   CTR_AMT  OTSND_BAL_AMT  \
0  6192662816516780923 2016-07-31     59.0  100000.0       93000.42   
1  6192662816516780923 2016-08-31     89.0  100000.0       94526.35   
2  6192662816516780923 2016-09-30    117.0  100000.0       89833.92   
3  6192662816516780923 2016-07-31     59.0  100000.0       93000.42   
4  6192662

In [24]:
df = prep.loc[(prep['POSN_DT'] < prep['FileDate']) & (prep['POSN_DT'] >= prep['Previous_FileDate'])]
print df.head()
print("Merge coverage")
print(1.0*len(df.index)/len(target.index))

              HASH_AR_ID   FileDate Previous_FileDate Next_FileDate  \
30  -5086591535661867418 2016-08-04        2016-07-07    2016-09-06   
51   -389919671788712216 2016-08-04        2016-07-07    2016-09-06   
56   -389919671788712216 2016-10-06        2016-09-06           NaT   
84   8810541145679989509 2016-08-04        2016-07-07    2016-09-06   
88   8810541145679989509 2016-09-06        2016-08-04    2016-10-06   

             HASH_IP_ID    POSN_DT  DLQ_DYS   CTR_AMT  OTSND_BAL_AMT  \
30  6192662816516780923 2016-07-31     59.0  100000.0       93000.42   
51 -1623440567072454905 2016-07-31      0.0  200000.0      183845.75   
56 -1623440567072454905 2016-09-30      0.0  200000.0       51723.20   
84 -1708046059400153119 2016-07-31     29.0  120000.0       68141.75   
88 -1708046059400153119 2016-08-31     59.0  120000.0       69313.60   

    ACR_INT_AMT    PNP_AMT  EFF_INT_RATE AUTO_DB_F  PRVN_CLT_AMT  INT_ARS_AMT  \
30      3166.50   89833.92          20.0         3         

In [25]:
### new feature ###
df_new = pd.DataFrame({'NBR_LN_AR':df.groupby( ['HASH_IP_ID','FileDate'] )['HASH_AR_ID'].nunique(),
                     'TOTAL_LN_AMT':df.groupby( ['HASH_IP_ID','FileDate'] )['OTSND_BAL_AMT'].sum(),
                   'TOTAL_DLQ_DYS':df.groupby( ['HASH_IP_ID','FileDate'] )['DLQ_DYS'].sum()}
                      ).reset_index()

idx = df.groupby(['HASH_IP_ID','FileDate'])['OTSND_BAL_AMT'].transform(max) == df['OTSND_BAL_AMT']
df = pd.merge(df[idx],df_new,on=['HASH_IP_ID','FileDate'],how='inner')


In [26]:
# dsp.write_table(df, 'sor_ln_ar_' + min_date_text + '_' + max_date_text, 'feature')

In [ ]:
# print(len(df.index))

In [ ]:
# dsp.data_directory

In [ ]:
df_train, df_test = filter_dates(df, min_date_train, max_date_train, min_date_test, max_date_test)

In [ ]:
len(df_train.index)

In [ ]:
len(df_test.index)

In [ ]:
dsp.write_table(pd.concat([df_train, df_test]), 'sor_ln_ar_' + min_date_text + '_' + max_date_text, 'feature')